<a href="https://colab.research.google.com/github/jiahfong/incoherent-thoughts/blob/develop/Information_Condensing_Active_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Information Condensing Active Learning

# Introduction

Acquisition functions usually use model predictions/point locations (i.e. input features or learned representation space) to decide which points will improve model accuracy.
> IDEA: does latent disentanglement + distance metric work well? Find latent representations with greatest distance according to some distance metric

**GOAL**: develop and active learning acquisition function to select points that maximise the _eventual test accuracy_. This minimises the uncertainty on the unlabelled set.
> Conspicuously did not mention the speed at which it reaches the desired test accuracy?

* Selecting points that maximise MI between model params and labels doesn’t always minimise the model’s uncertainty on the unlabelled pool. Paper will demonstrate this point
* Instead, ICAL searches for the batch that maximises statistical dependency between the model’s predictions on the batch (B) and the model’s predictions on the unlabelled pool (instead of model params).
* Greedy algorithm employed to search for such a batch B (cf. feature selection)
* HSIC instead of MI as MI is hard to approximate using just samples (Song & Ernon 2019)
* HSIC is also differentiable, applicable in areas where MI would be difficult to make work.

# Related work

## Bayesian active learning by disagreement (BALD)
MI between unlabelled labels and model parameters. Turns out to be the points where the models are individually confident but are disagreeing with one another.

## Guo & Schuurmans (2008)
Find a batch such that the post-acquisition model is confident on the training set and has low uncertainty on the unlabelled dataset. This implies retraining the model for every candidate batch (read: exponential!) and is infeasible for large neural nets (method was mainly used for logistic regression)

## BMDR
Selects points that are close to the decision boundary while maintaining the overall sample distribution. The overall sample distribution is measured using maximum mean discrepancy (MMD) on *input features* between candidate batch and the set of all points (low MMD implies more representative batch). This method requires a decision boundary to work.

## BMAL
Selects a batch such that the Fisher information matrix between the batch and the
unlabelled pool is as close as possible. Note, this matrix is quadratic in the number of parameters, thus infeasible for NNs.

## Filtered Active Subset Selection
1. Pick a subset of points $\mathcal{B'}$ from the unlabelled pool where the model is most uncertain. Uncertainty is quantified using entropy in this case: $H(y|x, D_{train})$. Recall, entropy is low if the models are disagreeing with each other (first term in BALD).
2. Select a subset of points in `1.` that are representative as a whole (similar to ICAL in this regard) which favours points that can represent diversity of the set in `1.`. In other words, find $\mathcal{B}$ such that $f(\mathcal{B}) = \displaystyle\sum_{y\in\mathcal{Y}}\displaystyle\sum_{i\in V^y}\text{max}_{s\in \mathcal{B} \cap V^y}w(i, s)$ where:
    - $V^y$ is the set of points in $\mathcal{B'}$ that has predicted label $y$. ($V^y \subseteq \mathcal{B}'$)
    - $w(i, s)$ is a similarity function (e.g. $w(i, s) = d - ||x_i - x_s||^2_2$ where $d$ is the maximum possible distance between any two points)


## BatchBALD
Extending BALD to account for acquisition size of > 1. Eliminates overcounting in BALD when candidate batch size is > 1.

## Bayesian Batch Active Learning as Sparse Subset Approximation
Adapts Bayesian Coreset approach. This approach changes the batch size for every acquisition.

## DeepFool
Uses the concept of advesarial examples to find points close to the decision boundary.
The distance between an example and one of its adversarial examples is used as an approximation of its distance to the current decision boundary (presumably a proportional relationship?). This approach mitigates the need to have an explicit decision boundary.



## Others
1. Sener & Savarese (2017) and FF-Comp (Geifman & El-Yaniv, 2017) frames the problem as a core-set selection problem
2. Discriminative Active Learning [DAL](https://openreview.net/pdf?id=rJl-HsR9KX) trains a classifier to distinguish between labelled and unlabelled. If the classifier is confident that the datapoint is unlabelled, then intuitively these points are most unlike the labelled points and thus should be informative.
3. [BADGE](https://arxiv.org/pdf/1906.03671.pdf) samples points which are high in magnitude and diverse in a hallucinated gradient space w.r.t. the last layer of a NN.

These methods are not model agnostic as the require access to learnt representational space.

TODO: find out how these methods work (esp. about learnt representational spaces)


# Background

The paper defines *dependency* $\delta$ between a set of random variables $X_{1:n}$ as follows:

$$
\delta(X_{1:n}) = \Lambda(P_{1:n}, \otimes_iP_i)
$$

where:

1. $P_{1:n}$ is the joint distribution of $X_{1:n}$
2. $P_i$ is the marginal distribution of $X_i$
3. $\otimes_i$ the product of marginals
4. $\Lambda$ a divergence function. If $\Lambda$ is KL divergence, then we get MI as the dependency measure ($\mathbb{I}[X, Y] = D_{KL}(P(x, y)\ ||\ P(x) P(y))$); if it was maximum mean discrepency (MMD), then the dependency measure is known as **Hilbert-Schmidt Independence Criterion** (HSIC).

To extend two-variable HSIC to arbitiarily many variables, the authors used [_dHISC_](https://arxiv.org/pdf/1603.00285.pdf)

# Motivation


The upshot of this section is that picking the points with the most amount of information w.r.t. __model parameters__, for example BALD, could in fact increase the uncertainty of prediction on unlabelled data. The paper gave a toy example to demonstrate this and suggests picking points with the most amount of information w.r.t. model's predictions on the __unlabelled points__.

# Information Condensing Active Learning (ICAL)

## Acquisition function

The authors defined the acquisition function as follows:
$$
\alpha_{ICAL}(\{x_1, \dots, x_B\}, \delta) = \frac{1}{|\mathcal{D}_U|}\displaystyle\sum_{x'\in\mathcal{D}_U} \delta(y_{x'}, \{y_{x_1}, \dots, y_{x_B}\})
$$

where $\mathcal{D}_U$ is the unlabelled pool and $\delta$ the dependency measure. (see above). 

TODO: figure out the conflicting use of notation: $\delta(X_{1:n})$ vs $\delta(y_{x'}, \{y_{x_1}, \dots, y_{x_B}\})$



## Scaling ICAL

There are two types of ICAL introduced: normal ICAL and ICAL-pointwise.
ICAL-pointwise turns out to be more time efficient and peformed slightly better in the early acquisitions.

## Larger batch sizes

The authors are aware that by scaling to larger batch sizes, they might sacrifice diversity in the batch.

> ICAL-pointwise can accommodate larger batch sizes compared to normal ICAL


# Results

The authors benchmarked a subset of the methods outlined in _Related work_, namely: BatchBALD, BALD, random acquisition, ICAL, BayesCoreset, FASS, and Max entropy.

## MNIST & Repeated MNIST

The results from ICAL, BatchBALD, and BayesCoreset are neck and neck.
(30 acquisitions; batch acquisition size = 10; 50 MC dropout samples)

## EMNIST
ICAL outperforms BatchBALD and BayesCoreset. 
(60 acquisitions; batch acquisition size = 5; 50 MC dropout samples)

The authors attributed ICAL's performance to its ability to acquire a more diverse and balanced set of batches while stating that the other methods have under and over represented classes. The figure (Figure 4) shows that BatchBALD, random acquisition, and Max Entropy completely missed some classes in `EMNIST` (47 classes)! (Probably a contrived example given random acquisition should on average have ~6 points per class).

The authors argues that ICAL is more robust even as the number of classes increases (47) whereas the other alternatives degenerate.

## Fashion MNIST
Likewise, ICAL outperforms the BatchBALD and BayesCoreset. **Interestingly, the runner-up for FashionMNIST is random acquisition, besting both BatchBALD and BayesCoreset.**

(30 acquisitions; batch acquisition size = 10; 100 MC dropout samples)

## CIFAR-10 & CIFAR-100
ICAL outperforms all others. BayesCoreset performs the closest to ICAL.

(10 acquisitions on CIFAR-10 and 7 acquisitions on CIFAR-100; batch acquisition size = 3,000; MC dropout samples _not mentioned_).

> BatchBALD ran out of memory on both datasets and was not benchmarked.




# Conclusion

ICAL is model agnostic, applicable to both classification and regression tasks, and scales well with large batch sizes and large unlabelled pool sizes.

Future work includes even larger batch sizes (exploring techniques used in feature selection) and a hybird of getting the most information for __both__ model parameters (like BALD) and labels of the unlabelled pool in a single acquisition function.

# Closing remarks (Relatability to AL project)

1. The methods in _related work_ is worth exploring as there are some interesting ideas about AL in the context of deep learning and the authors did not benchmark all of them.
2. Extend BatchBALD to work with larger batch sizes (presumably 3000 samples at once is much)
3. Explore the direction of _diversity_ in the samples acquired. Does diversity decrease as the number of classes increase, as the authors postulated?
4. Does dHISC really make a difference? What if we used MI as the dependecy measure $\delta$ instead of dHISC? (The authors cited papers that support their claim (Song & Ermon 2019)) It will no longer be ICAL but we could try taking the MI w.r.t. labels of the unlabelled pool as opposed to the usual model parameters.
5. The idea of a hybrid acquisition function seems appealing; can we combine information gained from labels of the unlabelled pool and model parameters together? Naively one could consider a weighted approach:
    - $\alpha$ * MI w.r.t. model params + $\beta$ * MI w.r.t. labels of unlabelled pool or
    - $\alpha$ * MI w.r.t. model params + $\beta$ * dHSIC w.r.t. labels of unlabelled pool or
    - $\alpha$ * dHISC w.r.t. model params + $\beta$ * MI w.r.t. labels of unlabelled pool or
    - $\alpha$ * dHISC w.r.t. model params + $\beta$ * dHISC w.r.t. labels of unlabelled pool for some $\alpha, \beta \in \mathbb{R}$

# Compilation of TODOs

1. Find out how the methods in _Related work_ work (esp. about learnt representational spaces).
2. Figure out the conflicting use of notation: $\delta(X_{1:n})$ vs $\delta(y_{x'}, \{y_{x_1}, \dots, y_{x_B}\})$.
3. (Understand and) Implement [dHSIC](https://arxiv.org/pdf/1603.00285.pdf)/ICAL.
4. Read [Song & Ermon (2019)](https://arxiv.org/pdf/1910.06222.pdf)